### Exemplo de cálculo e extração de atributos de EEG para um arquivo CSV
Fonte: https://github.com/Abpadfoot/EpilepsyDetection

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep
import os
import pywt
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
%matplotlib notebook

In [4]:
# Cria listas de arquivos contendo todos os sinais do diretório separados por épocas
dirA ="./Datasets/setB/"
tempA = []
for file in os.listdir(dirA):
    fl = dirA + file
    tempA.append(fl)
tempA = sorted(tempA)    # class: 1     val:  -1

dirD="./Datasets/setC/"
tempD = []
for file in os.listdir(dirD):
    fl = dirD + file
    tempD.append(fl)
tempD = sorted(tempD)   # class:2       val: 0
    
dirE="./Datasets/setE/"
tempE = []
for file in os.listdir(dirE):
    fl = dirE + file
    tempE.append(fl)
tempE = sorted(tempE)   # class: 3        val: 1

In [5]:
tempA

['./Datasets/setB/O001.txt',
 './Datasets/setB/O002.txt',
 './Datasets/setB/O003.txt',
 './Datasets/setB/O004.txt',
 './Datasets/setB/O005.txt',
 './Datasets/setB/O006.txt',
 './Datasets/setB/O007.txt',
 './Datasets/setB/O008.txt',
 './Datasets/setB/O009.txt',
 './Datasets/setB/O010.txt',
 './Datasets/setB/O011.txt',
 './Datasets/setB/O012.txt',
 './Datasets/setB/O013.txt',
 './Datasets/setB/O014.txt',
 './Datasets/setB/O015.txt',
 './Datasets/setB/O016.txt',
 './Datasets/setB/O017.txt',
 './Datasets/setB/O018.txt',
 './Datasets/setB/O019.txt',
 './Datasets/setB/O020.txt',
 './Datasets/setB/O021.txt',
 './Datasets/setB/O022.txt',
 './Datasets/setB/O023.txt',
 './Datasets/setB/O024.txt',
 './Datasets/setB/O025.txt',
 './Datasets/setB/O026.txt',
 './Datasets/setB/O027.txt',
 './Datasets/setB/O028.txt',
 './Datasets/setB/O029.txt',
 './Datasets/setB/O030.txt',
 './Datasets/setB/O031.txt',
 './Datasets/setB/O032.txt',
 './Datasets/setB/O033.txt',
 './Datasets/setB/O034.txt',
 './Datasets/s

In [7]:
ta=[] # Agrupa todos os sinais listados na lista ta
st = 'A'
for i in range(len(tempA)):
    x = pd.read_table(tempA[i],header=None)
    x.columns=[st+str(i)]
    ta.append(x)
    
td=[] # ... td
st = 'A'
for i in range(len(tempD)):
    x = pd.read_table(tempD[i],header=None)
    x.columns=[st+str(i)]
    td.append(x)
    
te=[] # ... tc
st = 'A'
for i in range(len(tempE)):
    x = pd.read_table(tempE[i],header=None)
    x.columns=[st+str(i)]
    te.append(x)

In [10]:
# Define função para concatenar colunas em uma grande tabela
def table(table):
    big_table = None
    for ta in table:
        big_table = pd.concat([big_table, ta],axis=1)
    return big_table

In [17]:
# Preenche a tabela
bigA = table(ta)
bigD = table(td)
bigE = table(te)
head = list(bigA.columns.values)

In [29]:
bigA

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A90,A91,A92,A93,A94,A95,A96,A97,A98,A99
0,-24,-55,-36,-14,-58,87,-52,2,8,-53,...,62,-128,-83,98,-53,29,-76,27,-6,-18
1,-22,-48,-40,-5,-78,98,-56,-6,0,-15,...,49,-158,-120,178,-33,10,-102,4,-19,-13
2,-17,-48,-36,0,-83,103,-49,-22,-16,11,...,14,-161,-123,236,-23,-26,-95,1,-47,0
3,-18,-38,-35,7,-69,108,-51,-17,-38,38,...,-14,-158,-119,237,0,-81,-65,19,-83,9
4,-19,-23,-22,7,-31,97,-53,6,-31,35,...,-14,-152,-93,187,10,-127,-30,43,-99,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,21,-93,70,98,-23,18,-31,92,-108,-110,...,-33,-20,-3,-21,94,-90,-9,98,-55,91
4093,23,-95,49,86,-36,17,-33,76,-83,-118,...,-39,-45,-5,-10,80,-90,-24,77,-67,89
4094,22,-84,37,76,-43,36,-37,59,-45,-103,...,-34,-79,-27,0,53,-86,-34,65,-62,52
4095,9,-60,20,57,-44,58,-44,39,0,-77,...,-24,-95,-50,-12,39,-88,-32,47,-43,-4


In [30]:
def creat_mat(mat):
    matx = np.zeros((len(mat),(len(head))))
    for i in range(len(head)):
        matx[:,i] = mat[head[i]]
#        sleep(0.01)
    return matx

In [33]:
matA = creat_mat(bigA) # : refers to healthy data
matD = creat_mat(bigD) # : refers to Inter-ictal (transition between healthy to seizure)
matE = creat_mat(bigE) # : of ictal or seizures

matA = np.nan_to_num(matA) # matB[:,0] --- > channel 0, matB[:,1] --- > channel 1 like that
matD = np.nan_to_num(matD)
matE = np.nan_to_num(matE)


# 4097 data point per channel 
# 173.61 Hz sample rate and there are 4097 data point for each channel
# total 100 channel are their
# 4097/173.61 = 23.59 sec 
# the raw data from one of the channels for the 23.59 sec

In [34]:
# Define número de features a serem calculadas e cria uma lista de nomes para colunas f1..f4
feature_size = 4
columns_name = list()
for i in range(feature_size):
    columns_name = columns_name + ['f'+str(i+1)]
columns_name = columns_name + ['class']

In [35]:
# Função para a extração de features.
# Descomentar as features desejadas e ajustar o número de features de acordo na célula anterior
def features(mat):
    Kmax = 5
    Tau  = 4
    DE   = 10
    M    = 10
    R    = 0.3
    Band = np.arange(1,86)
    Fs   = 173
    #fft = np.fft.fft(mat)
    #mat = np.fft.ifft(fft[0:1500])
    #mat = abs(np.fft.ifft(fft[0:1000]))
    lis = list()
    #lis = lis + [np.max(mat)]
    #ct = 0
    #ma = 0
    #mat1 = mat>75 
    #for i in range(len(mat1)-1):
    #    if(mat1[i]==mat1[i+1]):
    #        if(mat1[i]==1):
    #            ct = ct+1
    #            ma = max(ma,ct)
    #        else: 
    #            ct = 0
    #lis = lis + [ma]
    #lis = lis + [pyeeg.dfa(mat)]                 #DFA
    #lis = lis + [pyeeg.hfd(mat,Kmax)]            #HFD 
    #lis = lis + [pyeeg.svd_entropy(mat,Tau,DE)]  #SVD_Entropy
    #lis = lis + [pyeeg.fisher_info(mat,Tau,DE)]  #Fisher_Information
    #lis = lis + [pyeeg.pfd(mat)]                 #PFD
    #lis = lis + [np.mean(pywt.wavedec(mat,'db2',level=2)[0])]
    #lis = lis + [np.min(pywt.wavedec(mat,'db2',level=2)[0])]
    #lis = lis + [np.max(pywt.wavedec(mat,'db2',level=2)[0])]
    lis = lis + [np.std(pywt.wavedec(mat,'db4',level=8)[0])]
    #lis = lis + [np.mean(pywt.wavedec(mat,'db2',level=2)[1])]
    #lis = lis + [np.min(pywt.wavedec(mat,'db2',level=2)[1])]
    #lis = lis + [np.max(pywt.wavedec(mat,'db4',level=8)[3])]
    lis = lis + [np.std(pywt.wavedec(mat,'db4',level=8)[4])]
    #lis = lis + [np.mean(pywt.wavedec(mat,'db2',level=2)[2])]
    #lis = lis + [np.min(pywt.wavedec(mat,'db2',level=2)[2])]
    #lis = lis + [np.max(pywt.wavedec(mat,'db2',level=2)[2])]
    lis = lis + [np.std(pywt.wavedec(mat,'db4',level=8)[5])]
    lis = lis + [np.std(pywt.wavedec(mat,'db4',level=8)[6])]
    #lis = lis + [np.std(pywt.wavedec(mat,'db4',level=8)[7])]
    
    #lis = lis + [np.max(abs(fft))]
    #lis = lis + [np.min(abs(fft))]
    #lis = lis + [np.mean(abs(fft))]
    #lis = lis + [np.std(abs(fft))]
    #lis = lis + [np.max(mat)]
    #lis = lis + [np.min(mat)]
    #lis = lis + [np.mean(mat)]
    #lis = lis + [np.std(mat)]
    sleep(0.01)
             
    return lis

In [36]:
MftA = np.zeros((100,feature_size + 1))

for i in range(100):
    MftA[i,:] = features(matA[:,i]) + [1]

In [37]:
MftD = np.zeros((100,feature_size + 1))

for i in range(100):
    MftD[i,:] = features(matD[:,i]) + [0]

In [38]:
MftE = np.zeros((100,feature_size + 1))

for i in range(100):
    MftE[i,:] = features(matE[:,i]) + [-1]

In [39]:
FCM_A = pd.DataFrame(MftA,columns=columns_name)
FCM_D = pd.DataFrame(MftD,columns=columns_name)
FCM_E = pd.DataFrame(MftE,columns=columns_name)
#FCM_B.head(4)

In [40]:
TotalDataset = pd.concat([FCM_A,FCM_D,FCM_E],ignore_index=True)

In [41]:
TotalDataset.to_csv(index=False)

'f1,f2,f3,f4,class\r\n262.68705113441035,94.97760670614608,96.48426614128628,65.3225232090065,1.0\r\n364.2729328040349,94.49869946257576,95.6538582393757,68.96792849917232,1.0\r\n292.00483546831686,96.50637432699733,100.8456830513739,85.33939015349371,1.0\r\n314.57652202198244,105.16111218431958,133.99250714635843,101.09124103228406,1.0\r\n388.0194009541097,64.71610200808554,132.62418410846539,94.98886289967417,1.0\r\n340.06491333297004,87.25655812968463,109.40001091350655,81.09145270323461,1.0\r\n335.9229686527583,151.66893415611713,134.46158034861983,99.47595950240648,1.0\r\n160.79847452335184,96.01294885526094,99.86826719776067,71.44266432470496,1.0\r\n299.3601386298178,97.48081061240181,110.45917868539318,82.76922283068384,1.0\r\n358.50995002451515,112.67930626265469,223.09584829720572,168.2949957473398,1.0\r\n845.5925215387917,129.86103685264294,258.6224339350575,191.126755782815,1.0\r\n181.7787892383776,65.40861566578381,97.42613646947584,83.46802820887612,1.0\r\n280.643916405336

In [43]:
TotalDataset.to_csv('features.csv') # Salva atributos extraídos